In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
# dataset

context_size = 3
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * context_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [ ]:
C = torch.randn((27, 2))

In [ ]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [ ]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [ ]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # in torch, -1 means the remaining shape, in this case - emb.shape[0]

In [ ]:
h, h.shape

(tensor([[ 0.2641, -0.2273, -0.8170,  ..., -0.9746, -0.9052, -0.9985],
         [-0.9976,  0.8819, -0.1612,  ..., -0.6095, -0.4565, -0.7046],
         [-0.8383,  0.9999,  0.2629,  ..., -0.9276, -0.9988,  0.7018],
         ...,
         [-0.7368, -0.0126,  0.7002,  ..., -0.9590, -0.9744,  0.9626],
         [-0.9985,  0.7671,  0.4630,  ...,  0.2473,  0.0629,  0.9999],
         [-0.7981,  0.9985,  0.5732,  ..., -0.9507, -0.1097,  0.9975]]),
 torch.Size([32, 100]))

In [ ]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [ ]:
logits = h @ W2 + b2

In [ ]:
logits, logits.shape

(tensor([[-4.3787e+00,  4.9812e+00, -6.0527e+00, -1.4244e+01, -5.1167e+00,
          -1.4158e+01, -2.1131e+00, -8.2833e+00,  9.5724e+00, -1.7202e+01,
          -3.0173e+00,  7.9378e+00, -9.9966e+00,  9.0600e+00,  1.2202e+01,
           3.7982e+00,  8.8386e+00,  5.2666e+00, -1.0417e+01,  1.5749e+01,
           3.9926e+00, -4.9464e+00,  4.5779e+00,  5.5415e+00, -1.6536e+00,
           2.7799e+00,  8.9110e+00],
         [ 4.8456e+00, -4.2982e+00, -1.2984e+01, -1.2517e+01, -8.2033e-01,
          -1.2055e+01,  6.1135e+00, -9.4829e+00,  1.0147e+01, -7.0363e+00,
          -1.5820e+01,  4.2797e+00, -6.5564e+00,  2.1729e+00,  1.5063e+01,
          -4.3221e+00,  5.7063e+00,  1.1432e+01, -3.0500e-02,  1.3963e+01,
           8.6780e+00,  1.7822e+01, -2.3849e+00,  2.8229e-01, -1.2710e+01,
          -7.7941e-01,  1.3337e+01],
         [ 1.3612e+01, -1.6215e+01, -7.1287e+00, -8.4374e+00, -5.8226e+00,
           1.2947e+01,  1.0124e+01,  1.6279e+00,  1.3802e+01,  5.2088e+00,
           6.8021e+00, -1.

In [ ]:
counts = logits.exp()

In [ ]:
prob = counts / counts.sum(1, keepdim=True)

In [ ]:
prob.shape

torch.Size([32, 27])

In [ ]:
prob[0].sum()

tensor(1.0000)

In [ ]:
loss = -prob[torch.arange(32), Y].log().mean(); loss

tensor(16.1433)

In [ ]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [ ]:
Y